In [1]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt

%matplotlib inline

#Root de archivos de datos
root_dir = '../data/'

#Lee menos data si es testing
is_testing = False

# Cantidad minima de aparicion de equipos en un dataframe
min_devices = 5

# Identificador unico de equipos
device_uuid = ['ref_hash']

# Ventanas de tiempo
windows = pd.DataFrame({
    'begin_date': [dt.datetime(2019,4,18),dt.datetime(2019,4,21),dt.datetime(2019,4,24)],
    'window_nr':[1,2,3]
})

# Armado de Features

Se utilizan 3 ventanas de tiempo y se procesan los dataframes para obtener features de esas 3 ventanas por separado. 

Como resultado se obtiene un dataset de features mas grande cuyos datos no se solapan en tiempo. Por lo que son validos para la prediccion. 

Se realiza la lectura y limpiado de los dataframes principales, con los mismos se realiza el filtrado y armado de los features. 

---
## 1. Lectura de DataFrames

In [2]:
#Optimizado para menos memoria
auction_dtypes = {
    'device_id': np.int64,
    'source_id': np.int8
}

auctions = pd.read_csv(root_dir + 'auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       usecols=list(auction_dtypes.keys()) + ['date'],
                       parse_dates = ['date'])
auctions.rename({'device_id':'ref_hash',
                 'date':'created'}, inplace=True, axis='columns')
auctions['n'] = 1

# Para hacer pruebas ocupando menos memoria se hace un sampleo aleatorio de la mitad del dataframe y se elimina el resto
if is_testing:
    auctions = auctions.sample(frac=.30)
    print("Is Testing, #records:", auctions['n'].sum())
else:
    print("Not Testing, #records:", auctions['n'].sum())

Not Testing, #records: 47409528


In [3]:
# Labels a submitir con las predicciones
to_predict = pd.read_csv(root_dir + 'target_competencia_ids.csv')
to_predict = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
to_predict.columns = ['ref_hash']

In [4]:
unique_hashes = pd.read_csv(root_dir+'unique_hashes.csv')

---
## 2. Arreglo de los datos

### Separo las semanas de entrenamiento

Se utiliza el siguiente metodo: 

1. Ventana del 18 al 20 inclusive (1) -> Predice valores entre el 21 y 24 (2)
2. Ventana del 21 al 23 inclusive (2) -> Predice valores entre el 24 y 26 (3)

In [5]:
#Agrego la ventana de tiempo
auctions = auctions.loc[auctions['ref_hash'].isin(unique_hashes['ref_hash'])]\
                   .sort_values(by='created')
auctions = pd.merge_asof(auctions,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

### 2.1 Arreglo de Auctions

In [6]:
%%time
#Elimino los registros con menos de un minimo de entradas, ya que no hay mucho que predecir en estos casos
group = ['ref_hash','window_nr']
orig_count = auctions['n'].sum()
auctions = auctions.groupby(group, sort = False)\
                   .filter(lambda x: x['n'].sum() >= min_devices)\
                   .sort_values(by=group+['created'])
last_count = auctions['n'].sum()
print('Eliminados:', orig_count-last_count)

Eliminados: 736875
CPU times: user 10min 7s, sys: 18.3 s, total: 10min 25s
Wall time: 10min 21s


In [7]:
%%time
auctions['next_date'] = auctions.groupby(group, as_index = False, sort=False)['created'].transform(lambda x: x.shift(-1))
auctions = auctions.loc[(~auctions['next_date'].isnull())]
auctions['secs_to_next'] = (auctions['next_date'] - auctions['created']).transform(lambda x: round(x.total_seconds()))

CPU times: user 17min 17s, sys: 17.2 s, total: 17min 34s
Wall time: 17min 35s


In [8]:
auctions.head()

,created,ref_hash,source_id,n,window_nr,next_date,secs_to_next
9146132,2019-04-19 19:40:28.465866,41863526108385,8,1,1,2019-04-20 02:52:26.892880,25918
11162626,2019-04-20 02:52:26.892880,41863526108385,3,1,1,2019-04-20 02:59:02.509230,396
11194917,2019-04-20 02:59:02.509230,41863526108385,3,1,1,2019-04-20 03:06:01.675788,419
11229465,2019-04-20 03:06:01.675788,41863526108385,3,1,1,2019-04-20 03:08:57.388160,176
11244486,2019-04-20 03:08:57.388160,41863526108385,3,1,1,2019-04-20 03:11:26.463903,149


---
## 3. Armado de Features

A continuacion se comienzan a extraer los distintos features que formaran el set de entrenamiento. 
Se cruza cada ventana con los labels que se desean predecir. 

Para entrenar el set debe decidirse como utilizar los datos de las distintas ventanas. 

Se tomaran los datos en 2 ventanas diferentes y se validaran con dos ventanas diferentes, de esta forma no deberia haber problemas de solapamiento y se maximiza la cantidad de datos.

Para el set final puede armarse un set de labels con los valores de los ultimos 3 dias.

In [ ]:
max_dates = pd.DataFrame({'w_max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,24),dt.datetime(2019,4,27)], 'window_nr' : [1,2,3]})

In [61]:
temp = unique_hashes
temp['window_nr'] = 1
devices = temp.copy()
temp['window_nr'] = 2
devices = devices.append(temp)
temp['window_nr'] = 3
devices = devices.append(temp)

In [62]:
devices['window_nr'].value_counts()

3    662110
2    662110
1    662110
Name: window_nr, dtype: int64

In [63]:
devices = devices.set_index(group)

In [64]:
sample = auctions.head(100)

In [65]:
sample.head()

,created,ref_hash,source_id,n,window_nr,next_date,secs_to_next
9146132,2019-04-19 19:40:28.465866,41863526108385,8,1,1,2019-04-20 02:52:26.892880,25918
11162626,2019-04-20 02:52:26.892880,41863526108385,3,1,1,2019-04-20 02:59:02.509230,396
11194917,2019-04-20 02:59:02.509230,41863526108385,3,1,1,2019-04-20 03:06:01.675788,419
11229465,2019-04-20 03:06:01.675788,41863526108385,3,1,1,2019-04-20 03:08:57.388160,176
11244486,2019-04-20 03:08:57.388160,41863526108385,3,1,1,2019-04-20 03:11:26.463903,149


### Tiempo promedio de arribos

Se desea saber el tiempo promedio entre arribos de los dispositivos a las encuestas.

In [66]:
%%time
time = auctions.groupby(group,sort=False)['secs_to_next'].mean().rename('secs_to_next_mean')

CPU times: user 2.34 s, sys: 1 s, total: 3.34 s
Wall time: 3.34 s


In [67]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [68]:
devices.head()

,,secs_to_next_mean
ref_hash,window_nr,
2564673204772915246,1,764.662037
4441121667607578179,1,111.363636
7721769811471055264,1,281.286344
6416039086842158968,1,NaN
1258642015983312729,1,8364.666667


### Tiempo desde ultima aparicion

In [69]:
%%time
time = auctions.groupby(group,sort=False)\
               .tail(1)\
               .merge(max_dates, on='window_nr', how='left')\
               .set_index(group)

time = (time['w_max_date'] - time['created']).transform(lambda x: round(x.total_seconds())).rename('secs_since_last_arrival')

CPU times: user 13.9 s, sys: 2.26 s, total: 16.2 s
Wall time: 16.2 s


In [70]:
time.head()

ref_hash         window_nr
41863526108385   1             70206
135153013040192  1             70659
161514654074162  1            248758
186034136943920  1             74393
                 2             37583
Name: secs_since_last_arrival, dtype: int64

In [71]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [72]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival
ref_hash,window_nr,,
2564673204772915246,1,764.662037,88625.0
4441121667607578179,1,111.363636,29052.0
7721769811471055264,1,281.286344,333.0
6416039086842158968,1,NaN,NaN
1258642015983312729,1,8364.666667,121224.0


### Cantidad de apariciones en encuestas en la ventana

In [73]:
%%time
time = auctions.groupby(group ,sort=False)['n'].sum().rename('auctions_total')

CPU times: user 2.24 s, sys: 1.14 s, total: 3.39 s
Wall time: 3.4 s


In [74]:
time.head()

ref_hash         window_nr
41863526108385   1            34
135153013040192  1             7
161514654074162  1             5
186034136943920  1             6
                 2            58
Name: auctions_total, dtype: int64

In [75]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [76]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total
ref_hash,window_nr,,,
2564673204772915246,1,764.662037,88625.0,216.0
4441121667607578179,1,111.363636,29052.0,1188.0
7721769811471055264,1,281.286344,333.0,908.0
6416039086842158968,1,NaN,NaN,NaN
1258642015983312729,1,8364.666667,121224.0,12.0


### Cantidad de apariciones en encuestas en la ultima hora

In [88]:
%%time
time = auctions.merge(max_dates, on='window_nr', how='left')
time = time.loc[time['created'] >= (time['w_max_date'] - timedelta(hours=1))]\
           .groupby(group, sort=False)['n'].sum()\
           .rename('auctions_last_hour')

CPU times: user 14.7 s, sys: 10.4 s, total: 25.1 s
Wall time: 27.1 s


In [89]:
time.head()

ref_hash         window_nr
345999128501141  2             3
360710529886978  3             2
416301579449694  1             2
686608884458246  3            11
717556230663455  1            74
Name: auctions_last_hour, dtype: int64

In [90]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [91]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour
ref_hash,window_nr,,,,
2564673204772915246,1,764.662037,88625.0,216.0,NaN
4441121667607578179,1,111.363636,29052.0,1188.0,NaN
7721769811471055264,1,281.286344,333.0,908.0,23.0
6416039086842158968,1,NaN,NaN,NaN,NaN
1258642015983312729,1,8364.666667,121224.0,12.0,NaN


---
## 4. Guardo Features 

In [92]:
devices.to_csv('tmp.csv',header=True,index=True)